In [1]:
import os
import shutil
import glob
import json
import random
import scipy.io as sio
import pandas as pd

In [3]:
raw_data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri'

bids_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'

subnums = ['601', '609', '611', '619', '621', '629']

sessions = ['01', '02', '03']

data_types = ['anat', 'func']

misc_files = ['dataset_description.json', 'participants.json', 'README', 'CHANGES']

In [ ]:
from bidsify_helpers import make_bids_dirs, add_misc_bids_files

In [ ]:
make_bids_dirs()

In [ ]:
add_misc_bids_files()

In [4]:
timing_mats = glob.glob(raw_data_path + '/*/*/*.mat')

In [5]:
timing_mats[1]

'/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri/sub-609/ses-02/taskYN_subj609_day7_session4.mat'

In [33]:
tmp = sio.loadmat(timing_mats[1], squeeze_me=True)

In [34]:
tmp.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mydata', 'timing'])

In [35]:
timing = tmp['timing']

In [36]:
timing.dtype

dtype([('Begin', 'O'), ('onset', 'O'), ('feedbackOn', 'O'), ('crossStart', 'O'), ('crossEnd', 'O'), ('End', 'O'), ('session', 'O')])

In [37]:
len(timing.dtype)

7

In [105]:
# timing_keys = ['Begin', 'onset', 'feedbackOn', 'crossStart', 'crossEnd', 'End']
timing_keys = ['Begin', 'onset', 'feedbackOn', 'crossStart', 'crossEnd', 'End', 'session'] #session only exists for the later YN run
timing_vals = timing.item()

tmp_dict = dict(zip(timing_keys, timing_vals))
cut_dict = dict()
for k in timing_keys:
    half_len = int(len(tmp_dict[k])/2)
    cut_dict[k] = tmp_dict[k][-half_len:]

In [106]:
# cut_dict

In [107]:
# pd.melt(pd.DataFrame(dict(zip(timing_keys, timing_vals)))).drop_duplicates().sort_values(by=['value'])

run_begin = cut_dict['Begin']
run_end = cut_dict['End'] - cut_dict['Begin'] 
del cut_dict['Begin']
del cut_dict['End']
del cut_dict['session']

In [108]:
run_begin

array([5918.289145])

In [109]:
run_end

array([832.0858695])

In [110]:
timing_df = pd.DataFrame(cut_dict) - run_begin[0]

timing_df

,onset,feedbackOn,crossStart,crossEnd
0,10.016965,12.395206,-5918.289145,10.016927
1,14.201438,15.456594,13.430895,14.201429
2,16.998732,18.612107,16.481977,16.998724
3,23.693180,25.060359,19.636746,23.693156
4,26.607634,27.647158,26.090756,26.607626
...,...,...,...,...
195,809.000927,817.811072,806.280660,809.000918
196,819.357981,820.544537,818.842362,819.357973
197,822.093865,823.226760,821.577397,822.093857
198,826.078298,827.908183,824.262851,826.078289


In [115]:
run_end[0]

832.0858695000006

In [124]:
# np.array(timing_df['feedbackOn']) 
timing_df['crossEnd'][1:] - timing_df['crossStart'][1:]
# np.array(timing_df['crossStart'], run_end[0])

1      0.770535
2      0.516747
3      4.056410
4      0.516870
5      2.285334
         ...   
195    2.720258
196    0.515611
197    0.516459
198    1.815438
199    0.516117
Length: 199, dtype: float64

In [127]:

stim_timing = pd.DataFrame({'onset': timing_df['onset'], 'duration':  timing_df['feedbackOn'] - timing_df['onset'], 'trial_type': 'stim'})
feedback_timing = pd.DataFrame({'onset': timing_df['feedbackOn'], 'duration':  np.concatenate([np.array(timing_df['crossStart'][1:]), run_end]) - timing_df['feedbackOn'], 'trial_type': 'feedback'})
cross_timing = pd.DataFrame({'onset': timing_df['crossStart'][1:], 'duration': timing_df['crossEnd'][1:] - timing_df['crossStart'][1:], 'trial_type': 'fixCross'})
run_events = pd.concat([stim_timing, feedback_timing, cross_timing]).sort_values(by=['onset']).reset_index(drop=True)

run_events.to_csv('example.tsv', sep="\t")

,onset,duration,trial_type
0,10.016965,2.378240,stim
1,12.395206,1.035689,feedback
2,13.430895,0.770535,fixCross
3,14.201438,1.255156,stim
4,15.456594,1.025383,feedback
...,...,...,...
594,826.078298,1.829885,stim
595,827.908183,1.026348,feedback
596,828.934531,0.516117,fixCross
597,829.450657,1.625201,stim
